# Instalation package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python --version

In [ ]:
# !nvidia-smi
!dpkg -l | grep cudnn
!nvcc --version

In [ ]:
!apt-get update

In [ ]:
# !git clone https://github.com/opencv/opencv
# !git clone https://github.com/opencv/opencv_contrib
# !mkdir /content/build

# %cd /content/build

# !cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules \
#        -DBUILD_SHARED_LIBS=OFF \
#        -DBUILD_TESTS=OFF \
#        -DBUILD_PERF_TESTS=OFF \
#        -DBUILD_EXAMPLES=OFF \
#        -DWITH_OPENEXR=OFF \
#        -DWITH_CUDA=ON \
#        -DWITH_CUBLAS=ON \
#        -DWITH_CUDNN=ON \
#        -DOPENCV_DNN_CUDA=ON \
#        /content/opencv

# !make -j8 install

In [ ]:
!cp "/content/drive/MyDrive/DetectronFolder/build/lib/python3/cv2.cpython-310-x86_64-linux-gnu.so" .

In [ ]:
import cv2 as cv
print("CUDA supported:", cv.cuda.getCudaEnabledDeviceCount())

In [ ]:
!pip install cupy-cuda11x

In [ ]:
!pip install -U labelme2coco

In [ ]:
!pip install cucim

In [ ]:
!pip install GPUtil

# Generate Images

In [ ]:
!pwd
%cd /content/

In [ ]:
import math
from PIL import Image
import cupy as cp
import numpy as np
# import numpy as cp
import random
import json
import time
import os
import threading
import cv2 as cv
# import cucim
from skimage import io

class GenerateData:
    def __init__(self,img_size, chip_low , chip_high , stage_mid_color , grid_mid_color , wafer_mid_color ,
                stage_random , wafer_random , grid_random , pitch_x , pitch_y , scribe_x , scribe_y , ):
        self.img_size = img_size
        self.img_margin = 0.2*(self.img_size / 2)
        self.stage_margin = 0.15*(self.img_size/2)
        self.chip_low = chip_low
        self.chip_high = chip_high
        self.pitch_x = pitch_x
        self.pitch_y = pitch_y
        self.scribe_x = scribe_x
        self.scribe_y = scribe_y
        self.imgarr = cp.zeros((img_size,img_size), dtype=np.uint8)
        self.dim_x = self.imgarr.shape[0]
        self.dim_y = self.imgarr.shape[1]
        self.center_x = math.floor(self.dim_x / 2)
        self.center_y = math.floor(self.dim_y / 2)
        self.stage_radius = self.img_size/2 - self.stage_margin
        self.wafer_radius = self.img_size/2 - self.img_margin
        self.notch_radius = round(1.25*(self.wafer_radius/100))
        self.grid_radius = 0.95*self.wafer_radius
        self.stage_color = stage_mid_color
        self.grid_color = grid_mid_color
        self.wafer_color = wafer_mid_color
        self.stage_random = stage_random
        self.wafer_random = wafer_random
        self.grid_random = grid_random
        self.boundary_box = []

    def GenerateJson(self , name , topr_x , topr_y , botr_x , botr_y):
        self.boundary_box.append({
            "label": f"{name}",
            "points": [[int(topr_x), int(topr_y)], [int(botr_x), int(botr_y)]],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        })

    def Create_Stage(self):
        x1 = 0
        y1 = 0
        variation = math.floor(self.stage_random/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:
                if ((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.stage_radius ** 2):
                    self.imgarr[x1][y1] = self.stage_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Wafer_Edge(self):
        x1 = 0
        y1 = 0
        variation = math.floor(self.wafer_random/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:
                if ((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2):
                    self.imgarr[x1][y1] = self.wafer_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Notch(self):
        x1 = 0
        y1 = 0
        variation = math.floor(self.stage_random/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:
                if ((x1-(self.center_x + self.wafer_radius)) ** 2 + (y1-self.center_y) ** 2) <= (self.notch_radius ** 2):
                    self.imgarr[x1][y1] = self.stage_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Grid(self):
        half_scribe_x = math.floor(self.scribe_x/2)
        half_scribe_y = math.floor(self.scribe_y/2)
        xi = 0
        x=0
        x_i = 0
        y=0
        y_i = 0
        variation = math.floor(self.grid_random/2)
        for x1 in self.imgarr:
            y = 0
            y_i = 0
            for y1 in x1:
                if (x == x_i):
                    #print("found_x")
                    xi = 1
                    if ((x-self.center_x) ** 2 + (y-self.center_y) ** 2) <= (self.grid_radius ** 2):
                        #print("found")
                        temp_x = x-half_scribe_x
                        while temp_x <= x+half_scribe_x:
                            self.imgarr[temp_x][y] = self.grid_color + random.randint(-variation, variation)
                            temp_x += 1
                if(y == y_i):
                    #print("found_y")
                    y_i += self.pitch_y
                    if ((x-self.center_x) ** 2 + (y-self.center_y) ** 2) <= (self.grid_radius ** 2):
                        #print("found")
                        temp_y = y-half_scribe_y
                        while temp_y <= y+half_scribe_y:
                            self.imgarr[x][temp_y] = self.grid_color + random.randint(-variation, variation)
                            temp_y += 1
                y += 1
            x += 1
            if(xi):
                x_i += self.pitch_x
                xi = 0

    def create_img(self):
        self.Create_Stage()
        self.Create_Wafer_Edge()
        self.Create_Grid()
        self.Create_Notch()
        print("Finish Wafer")

    def point_in_circle(self , center_x , center_y , radius):
        while True:
            x = cp.random.randint(center_x - radius, center_x + radius + 1)
            y = cp.random.randint(center_y- radius, center_y + radius + 1)
            if (x - center_x)**2 + (y - center_y)**2 <= radius**2:
                return x, y

    def DrawBoundary(self ,min_x , min_y , max_x , max_y , name , min_box_size):
        if name != 'cloudy':
            width = abs(max_x - min_x)
            height = abs(max_y - min_y)
            if width < min_box_size:
                diff = min_box_size - width
                min_x -= diff // 2
                max_x += diff // 2
            if height < min_box_size:
                diff = min_box_size - height
                min_y -= diff // 2
                max_y += diff // 2
            self.GenerateJson(name=name , topr_x=min_x , topr_y= min_y , botr_x=max_x , botr_y= max_y)
            print(f"Finish Boundary for {name}")
        else:
            self.GenerateJson(name=name , topr_x=min_x , topr_y= min_y , botr_x=max_x , botr_y= max_y)
            print(f"Finish Boundary for {name}")

    def GenerateScratch(self, name, min_box_size, variance_add, scratch_variance, scratch_length, scratch_thickness,):
        x1, y1 = self.point_in_circle(self.center_x, self.center_y, self.grid_radius) # Check point in the circle
        angle = cp.random.uniform(0, 2 * np.pi) # Generate a random

        scratch_thickness = scratch_thickness - 1

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

        # Compute the x and y offsets using the angle and desired scratch length
        x2 = x1 + int(scratch_length * cp.cos(angle))
        y2 = y1 + int(scratch_length * cp.sin(angle))

        x = cp.linspace(x1, x2, scratch_length, dtype=int) # Generate x array from start to end within the length
        y = cp.linspace(y1, y2, scratch_length, dtype=int) # Generate y array from start to end within the length

        check_coords = [(ix, iy) for ix in range(-scratch_thickness, scratch_thickness + 1)
                            for iy in range(-scratch_thickness, scratch_thickness + 1)
                        if cp.sqrt(ix**2 + iy**2) <= scratch_thickness]


        for ix, iy in check_coords:
            x_check_coords = x + ix
            y_check_coords = y + iy
            valid_coords = (x_check_coords >= 0) & (x_check_coords < self.img_size) & (y_check_coords >= 0) & (y_check_coords < self.img_size) & (((x_check_coords-self.center_x) ** 2 + (y_check_coords-self.center_y) ** 2) <= (self.wafer_radius ** 2))
            x_check_coords, y_check_coords = x_check_coords[valid_coords], y_check_coords[valid_coords]

        for i in range(max(len(x_check_coords), len(y_check_coords))):
            if(variance_add == 2):
                self.imgarr[x_check_coords[i], y_check_coords[i]] = self.imgarr[x_check_coords[i], y_check_coords[i]] + random.randint(math.floor(scratch_variance/2), scratch_variance)
            else:
                self.imgarr[x_check_coords[i], y_check_coords[i]] = self.imgarr[x_check_coords[i], y_check_coords[i]] - random.randint(math.floor(scratch_variance/2), scratch_variance)
            found = 1
            min_x = min(min_x, y_check_coords[i])
            max_x = max(max_x, y_check_coords[i])
            min_y = min(min_y, x_check_coords[i])
            max_y = max(max_y, x_check_coords[i])

        # Boundary Box
        #min_x = min(x) - scratch_thickness
        #min_y = min(y) - scratch_thickness
        #max_x = max(x) + scratch_thickness
        #max_y = max(y) + scratch_thickness
        if found == 1:
            self.DrawBoundary(min_x, min_y, max_x, max_y, name, min_box_size)

    def Cloudy_defect(self, min_r, max_r, set_radius_x, set_radius_y, min_box_size, name, blurriness, cloudy_variance, variance_add, cloudy_boundary, check_inside):
        x_radius = set_radius_x or random.randint(min_r, max_r)
        y_radius = set_radius_y or random.randint(min_r, max_r)

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

        if (x_radius > self.wafer_radius or y_radius > self.wafer_radius):
            return 0

        while True:
            rand_x = random.randint(int(self.center_x - self.wafer_radius), int(self.center_x + self.wafer_radius))
            rand_y = random.randint(int(self.center_y - self.wafer_radius ), int(self.center_y + self.wafer_radius ))

            if(check_inside == True):
                break

            #Check if the entire ellipse lies within the wafer_radius
            #if all((((rand_x + dx) - self.center_x) ** 2 + ((rand_y + dy) - self.center_y) ** 2) <= (self.wafer_radius-cloudy_boundary) ** 2
            #for dx in [-x_radius, x_radius] for dy in [-y_radius, y_radius]):

            if((rand_x - self.center_x)**2 + (rand_y - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                if((rand_x - self.center_x)**2 + (rand_y + y_radius - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                    if((rand_x + x_radius - self.center_x)**2 + (rand_y + y_radius - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                        if((rand_x + x_radius - self.center_x)**2 + (rand_y - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                            break

        mask = cp.zeros_like(self.imgarr)
        # Draw a ellipse on the mask.
        #cv.ellipse(mask, (rand_x, rand_y), (x_radius, y_radius), 0, 0, 360, 255, -1)
        cv.rectangle(cp.asnumpy(mask), (rand_x, rand_y), (rand_x + x_radius, rand_y + y_radius), 255, -1)
        cp.asarray(mask)
        x1 = 0
        y1 = 0
        for x in mask:
            y1 = 0
            for y in x:
                if (not (((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2))):
                    mask[x1, y1] = 0
                elif((((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2))):
                    if(mask[x1,y1] == 255):
                        if(variance_add == 2):
                            self.imgarr[x1, y1] = self.imgarr[x1, y1] + random.randint(math.floor(cloudy_variance/2), cloudy_variance)
                            if(self.imgarr[x1, y1] > 255):
                                self.imgarr[x1, y1] = 255
                            elif(self.imgarr[x1, y1] < 0):
                                self.imgarr[x1, y1] = 0

                        else:
                            self.imgarr[x1, y1] = self.imgarr[x1, y1] - random.randint(math.floor(cloudy_variance/2), cloudy_variance)
                            if(self.imgarr[x1, y1] > 255):
                                self.imgarr[x1, y1] = 255
                            elif(self.imgarr[x1, y1] < 0):
                                self.imgarr[x1, y1] = 0

                        found = 1
                        min_x = min(min_x, y1)
                        max_x = max(max_x, y1)
                        min_y = min(min_y, x1)
                        max_y = max(max_y, x1)

                y1 = y1 +1
            x1 = x1 + 1

        if(found == 0):
            return 0

        if(blurriness % 2 == 0):
            blurriness += 1
        # Apply the Gaussian blur
        for i in range(1,blurriness,2):
            blurred_image = cv.cuda.createGaussianFilter(self.imgarr, (i, i), 0)

        # Combine original and blurred images based on the mask.
        self.imgarr = cv.cuda.bitwise_and(self.imgarr, self.imgarr, mask=~mask)
        blurred_region = cv.cuda.bitwise_and(blurred_image, blurred_image, mask=mask)
        self.imgarr = cv.cuda.add(self.imgarr, blurred_region)

        # Draw the bounding box.
        #box_width = max(2 * x_radius, min_box_size)
        #box_height = max(2 * y_radius, min_box_size)
        #top_left = (int(rand_x - box_width / 2), int(rand_y - box_height / 2))
        #bottom_right = (int(rand_x + box_width / 2), int(rand_y + box_height / 2))
        if found == 1:
            min_x = min_x -5
            min_y = min_y -5
            max_x = max_x +5
            max_y = max_y +5
            self.DrawBoundary(min_x , min_y , max_x, max_y, name, min_box_size)
            return 1
        return 0

    def GenerateSpotDefects(self, name, min_box_size, spot_variance, spot_radius, variance_add):
        x, y = self.point_in_circle(self.center_x, self.center_y, self.grid_radius)

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

    # Now, create a spot defect at the given x, y with the desired radius.
        for dx in range(-spot_radius, spot_radius + 1):
            for dy in range(-spot_radius, spot_radius + 1):
                if dx**2 + dy**2 <= spot_radius**2:  # This ensures that we only draw inside the circle.
                    x_coord, y_coord = x + dx, y + dy
                    if (0 <= x_coord < self.img_size) and (0 <= y_coord < self.img_size):  # Ensure the coordinates are inside the image.
                        if(((x_coord-self.center_x) ** 2 + (y_coord-self.center_y) ** 2) <= (self.wafer_radius ** 2)):
                            if(variance_add == 2):
                                self.imgarr[x_coord][y_coord] = self.imgarr[x_coord][y_coord] + random.randint(math.floor(spot_variance/2), spot_variance)
                            else:
                                self.imgarr[x_coord][y_coord] = self.imgarr[x_coord][y_coord] - random.randint(math.floor(spot_variance/2), spot_variance)
                            found = 1
                            min_x = min(min_x, y_coord)
                            max_x = max(max_x, y_coord)
                            min_y = min(min_y, x_coord)
                            max_y = max(max_y, x_coord)
        # Boundary Box
        if found == 1:
            self.DrawBoundary(min_x , min_y , max_x, max_y, name , min_box_size)

    def GenerateRadial(self, name, min_box_size, radius, angle1, variance, variance_add):
        angle1 = math.floor(angle1/2)
        x1, y1 = self.point_in_circle(self.center_x, self.center_y, self.grid_radius)
        angle_help = cp.arctan2((x1 - self.center_x), (y1 - self.center_y)) % (2 * cp.pi)
        angle_start = angle_help - ((angle1 * cp.pi)/180)
        angle_end = angle_help + ((angle1 * cp.pi)/180)

        # Create a grid of coordinates
        x, y = cp.meshgrid(cp.arange(self.img_size), cp.arange(self.img_size))
        # Calculate angles and radius for all points
        angles = cp.arctan2(x - x1, y - y1) % (2 * cp.pi)
        radiuses = (x - x1)**2 + (y - y1)**2

        # Find points within the specified angles and radius
        mask = (angle_start <= angles) & (angles <= angle_end) & (radiuses <= radius**2)

        # Apply changes to image array
        variance_values = cp.random.randint(math.floor(variance/2), variance, size=self.imgarr.shape)
        if variance_add == 2:
            temp_imgarr = self.imgarr + cp.where(mask, variance_values, 0)
        else:
            temp_imgarr = self.imgarr - cp.where(mask, variance_values, 0)

        # Clipping values to be within valid range and casting to original type
        cp.clip(temp_imgarr, 0, 255, out=temp_imgarr)
        self.imgarr = temp_imgarr.astype(self.imgarr.dtype)  # Cast back to original dtype

        # Update boundaries
        if cp.any(mask):
            indices = cp.argwhere(mask)
            min_x, min_y = cp.min(indices, axis=0).get()
            max_x, max_y = cp.max(indices, axis=0).get()
            self.DrawBoundary(min_x, min_y, max_x, max_y, name=name, min_box_size=min_box_size)
            return 1
        return 0


    def save_img_disk(self , filename , np_arr):
      cv.imwrite(filename, np_arr)

    def save_img(self , filename):
      # np_arr = cp.asnumpy(self.imgarr)
      # thr = threading.Thread(target = self.save_img_disk , args = (filename , np_arr))
      # thr.start()
      np_arr = cp.asnumpy(self.imgarr)
      cv.imwrite(filename , np_arr)

    def save_json(self , filename):
        data = {
            "version": "5.0.1",
            "flags": {},
            "shapes": self.boundary_box,
            "imagePath": filename,
            "imageData": None,
            "imageHeight": self.img_size,
            "imageWidth": self.img_size
        }
        json_file = os.path.join('drive/MyDrive/DetectronFolder/data/img_data/', os.path.basename(filename).split('.')[0] + '.json')
        with open(json_file, 'w') as json_file:
            json.dump(data, json_file)

###################################################################################


In [ ]:
import os
import shutil
import random
from labelme2coco import get_coco_from_labelme_folder, save_json

def clean_data(folder1, train_folder, test_folder):

    folders = [folder1, train_folder, test_folder]

    for folder in folders:
        if not os.path.exists(folder):
            try:
                os.mkdir(folder)
            except FileExistsError:
                pass

    all_jpg_files = [f for f in os.listdir(folder1) if f.endswith('.jpg')]
    print(len(all_jpg_files))
    train_count = int(len(all_jpg_files) * 0.8)   # 80%
    random.shuffle(all_jpg_files)
    for img in all_jpg_files[:train_count]:
        shutil.move(os.path.join(folder1 , img) , train_folder)
        corresponding_json = img.replace('.jpg', '.json')
        shutil.move(os.path.join(folder1, corresponding_json), train_folder)
    for img in all_jpg_files[train_count:]:
        shutil.move(os.path.join(folder1 , img) , test_folder)
        corresponding_json = img.replace('.jpg', '.json')
        shutil.move(os.path.join(folder1, corresponding_json), test_folder)

def convert_coco(foldername , name):
    if name == 'test':
        test_coco = get_coco_from_labelme_folder(foldername)

        save_json(test_coco.json, foldername+"test.json")
    else:
        train_coco = get_coco_from_labelme_folder(foldername)

        save_json(train_coco.json, foldername+"train.json")

In [ ]:
from PIL import Image
import numpy as np
import random


def save_base(gr , filename):
    gr.create_img()
    gr.save_img(filename)

if __name__ == '__main__':
    ##different paramters to change:

    number_of_images = 1
    number_of_reuse = 1 #how images will share the same background

    img_size= 601
    min_box_size = 20



    radial_defect_name = 'radial'
    spot_defect_name =  'spot'
    cloudy_defect_name = 'cloudy'
    line_defect_name = 'scratch'

    set_chip_color = None ##if you want to set the color directly
    set_stage_color = None ##if you want to set the color directly
    set_grid_color = None ##if you want to set the color directly
    set_grid_difference = None ## how much brighter or darker the grid is to the wafer
    set_grid_darker = None ##set to 1 if you want to make grid darker than wafer and 2 to make grid brighter otherwise random
    set_pitch_x = None ##if you want to set how big the grid is directly
    set_pitch_y = None ##if you want to set how big the grid is directly
    set_scribe_x = None ##if you want to set how thick the grid lines are
    set_scribe_y = None ##if you want to set how thick the grid lines are
    set_number_defects = None ##if you want to set how many defects per image
    set_cloudy_radius_x = None ##if you want to set the radius of the cloudy circle/oval
    set_cloudy_radius_y = None ##if you want to set the radius of the cloudy circle/oval
    set_cloudy_variance = None
    set_cloudy_variance_add = None ##set to 1 to make scratchs darker than base, 2 to make scratchs lighter than base
    set_cloudy_boundary = None ##how many pixels should the edge of any cloudy oval be from the wafer_edge
    set_scratch_length = None
    set_scratch_thickness = None
    set_scratch_variance = None
    set_scratch_variance_add = None ##set to 1 to make scratchs darker than base, 2 to make scratchs lighter than base
    set_spot_variance = None
    set_spot_radius = None
    set_spot_variance_add = None ##set to 1 to make spot darker than base, 2 to make scratchs lighter than base
    set_radial_radius = None
    set_radial_variance = None
    set_radial_angle = None
    set_radial_variance_add = None ##set to 1 to make radial darker than base, 2 to make scratchs lighter than base

    defect_min = 1
    defect_max = 5

    cloudy_check_inside = False ## set to true to check whether the entire clody rectangle is inside the wafer or not

    chip_low = 70
    chip_high = 200
    stage_low = 20
    stage_high = 40
    min_grid_difference = 10
    max_grid_difference = 20
    pitch_min = 10
    pitch_max = 15
    scribe_min = 1
    scribe_max = 3
    stage_variation_low = 5
    stage_variation_high = 10
    wafer_variation_low = 5
    wafer_variation_high = 10
    grid_variation_low = 2
    grid_variation_high = 5

    scratch_length_min = 10
    scratch_length_max = 20
    scratch_thickness_min = 1
    scratch_thickness_max = 2
    scratch_variance_min = 30
    scratch_variance_max = 40

    spot_radius_min = 1
    spot_radius_max = 3
    spot_variance_min = 20
    spot_variance_max = 30

    cloudy_radius_min = 5
    cloudy_radius_max = 20
    cloudy_blurriness = 5 ## should only be odd but no error just adds one
    min_cloudy_variance = 20
    max_cloudy_variance = 30
    min_cloudy_boundary = 3
    max_cloudy_boundary = 7

    radial_variance_min = 30
    radial_variance_max = 40
    radial_angle_min = 1
    radial_angle_max = 10
    radial_radius_min = 10
    radial_radius_max = 40


    #end of changeable parameters


    stage_mid_color = set_stage_color or random.randint(stage_low, stage_high)
    wafer_mid_color = set_chip_color or random.randint(chip_low, chip_high)
    grid_darker = set_grid_darker or random.randint(1,2)
    grid_difference = set_grid_difference or random.randint(min_grid_difference, max_grid_difference)
    if(grid_darker == 2):
        grid_mid_color = set_grid_color or (wafer_mid_color + grid_difference)
    else:
        grid_mid_color = set_grid_color or (wafer_mid_color - grid_difference)

    pitch_x = set_pitch_x or random.randint(pitch_min, pitch_max)
    pitch_y = set_pitch_y or random.randint(pitch_min, pitch_max)
    scribe_x = set_scribe_x or random.randint(scribe_min, scribe_max)
    scribe_y = set_scribe_y or random.randint(scribe_min, scribe_max)

    stage_random = random.randint(stage_variation_low, stage_variation_high)
    wafer_random = random.randint(wafer_variation_low, wafer_variation_high)
    grid_random = random.randint(grid_variation_low, grid_variation_high)


    gr = GenerateData(
        img_size,
        chip_low,
        chip_high,
        stage_mid_color,
        wafer_mid_color,
        grid_mid_color,
        stage_random,
        wafer_random,
        grid_random,
        pitch_x,
        pitch_y,
        scribe_x,
        scribe_y,

    )

    # save_base(gr , "base_img/baseimg.jpg")

    for i in range(number_of_images):

        start = time.time()

        if(i % number_of_reuse == 0):
            stage_mid_color = set_stage_color or random.randint(stage_low, stage_high)
            wafer_mid_color = set_chip_color or random.randint(chip_low, chip_high)
            grid_darker = set_grid_darker or random.randint(1,2)
            grid_difference = set_grid_difference or random.randint(min_grid_difference, max_grid_difference)
            if(grid_darker == 2):
                grid_mid_color = set_grid_color or (wafer_mid_color - grid_difference)
            else:
                grid_mid_color = set_grid_color or (wafer_mid_color + grid_difference)

            pitch_x = set_pitch_x or random.randint(pitch_min, pitch_max)
            pitch_y = set_pitch_y or random.randint(pitch_min, pitch_max)
            scribe_x = set_scribe_x or random.randint(scribe_min, scribe_max)
            scribe_y = set_scribe_y or random.randint(scribe_min, scribe_max)

            stage_random = random.randint(stage_variation_low, stage_variation_high)
            wafer_random = random.randint(wafer_variation_low, wafer_variation_high)
            grid_random = random.randint(grid_variation_low, grid_variation_high)

            gr = GenerateData(
            img_size,
            chip_low,
            chip_high,
            stage_mid_color,
            wafer_mid_color,
            grid_mid_color,
            stage_random ,
            wafer_random,
            grid_random,
            pitch_x,
            pitch_y,
            scribe_x,
            scribe_y,)

        gr.create_img()
        defect = set_number_defects or random.randint(defect_min, defect_max)

        done = 0

        for z in range(defect):

            cloudy_variance_add = set_cloudy_variance_add or random.randint(1,2)
            cloudy_variance = set_cloudy_variance or random.randint(min_cloudy_variance, max_cloudy_variance)
            cloudy_boundary = set_cloudy_boundary or random.randint(min_cloudy_boundary, max_cloudy_boundary)

            scratch_varince_add = set_scratch_variance_add or random.randint(1, 2)
            scratch_variance = set_scratch_variance or random.randint(scratch_variance_min, scratch_variance_max)
            scratch_length = set_scratch_length or random.randint(scratch_length_min, scratch_length_max)
            scratch_thickness = set_scratch_thickness or random.randint(scratch_thickness_min,scratch_thickness_max)

            spot_variance = set_spot_variance or random.randint(spot_variance_min, spot_variance_max)
            spot_radius = set_spot_radius or random.randint(spot_radius_min, spot_radius_max)
            spot_variance_add = set_spot_variance_add or random.randint(1,2)

            radial_radius = set_radial_radius or random.randint(radial_radius_min, radial_radius_max)
            radial_angle = set_radial_angle or random.randint(radial_angle_min, radial_angle_max)
            radial_variance = set_radial_variance or random.randint(radial_variance_min, radial_variance_max,)
            radial_variance_add = set_radial_variance_add or random.randint(1,2)

            random_defect = random.randint(1,4)
            if (random_defect == 1):
              done = gr.GenerateScratch(line_defect_name, min_box_size, scratch_varince_add, scratch_variance, scratch_length, scratch_thickness)
            if (random_defect == 2):
              done = gr.GenerateSpotDefects(spot_defect_name, min_box_size, spot_variance, spot_radius, spot_variance_add)
            if (random_defect == 3):
                done = gr.Cloudy_defect(cloudy_radius_min , cloudy_radius_max, set_cloudy_radius_x, set_cloudy_radius_y, min_box_size, cloudy_defect_name, cloudy_blurriness, cloudy_variance, cloudy_variance_add, cloudy_boundary, cloudy_check_inside)
            if (random_defect == 4):
                done = gr.GenerateRadial(radial_defect_name, min_box_size, radial_radius, radial_angle, radial_variance, radial_variance_add)
                # gr.GenerateRadialDefect2(min_box_size = min_box_size ,defect_length=random.randint(30, 50) , thickness=random.randint(1,5) , num_lines_per_defect=random.randint(2,5) , spacing=random.uniform(0.03 , 0.05) , line_height_scale=random.uniform(0.1,0.7))
            if (done == 0):
              defect = defect - 1

        end = time.time()

        print("Image takes " + f'{end - start} seconds')

        gr.save_img(filename= f'drive/MyDrive/DetectronFolder/data/img_data/test{i+1}.jpg')


        gr.save_json(filename= f'test{i+1}.jpg')


    ############ Clean Data #############

    # data_dir = 'drive/MyDrive/DetectronFolder/data/'
    # train_folder = data_dir + 'train/'
    # test_folder = data_dir + 'test/'

    # try:
    #     if not os.path.exists(data_dir):
    #       os.makedirs(data_dir)

    # except OSError:
    #     pass

    # clean_data(data_dir + 'img_data' , train_folder , test_folder)
    # convert_coco(train_folder , name= 'train')
    # convert_coco(test_folder , name='test')

# Main

In [ ]:
import GPUtil
if (GPUtil.getAvailable()):
    import cupy as np
    print("cupy")
else:
    import numpy as np
    print("numpy")
import math
import random
import cv2 as cv
import json
import os
import time

class GenerateData:
    def __init__(self,img_size, chip_low , chip_high , stage_mid_color , grid_mid_color , wafer_mid_color ,
                stage_random , wafer_random , grid_random , pitch_x , pitch_y , scribe_x , scribe_y , ):
        self.img_size = img_size
        self.img_margin = 0.2*(self.img_size / 2)
        self.stage_margin = 0.15*(self.img_size/2)
        self.chip_low = chip_low
        self.chip_high = chip_high
        self.pitch_x = pitch_x
        self.pitch_y = pitch_y
        self.scribe_x = scribe_x
        self.scribe_y = scribe_y
        self.imgarr = np.zeros((img_size,img_size), dtype=np.uint8)
        self.dim_x = self.imgarr.shape[0]
        self.dim_y = self.imgarr.shape[1]
        self.center_x = math.floor(self.dim_x / 2)
        self.center_y = math.floor(self.dim_y / 2)
        self.stage_radius = self.img_size/2 - self.stage_margin
        self.wafer_radius = self.img_size/2 - self.img_margin
        self.notch_radius = round(1.25*(self.wafer_radius/100))
        self.grid_radius = 0.95*self.wafer_radius
        self.stage_color = stage_mid_color
        self.grid_color = grid_mid_color
        self.wafer_color = wafer_mid_color
        self.stage_random = stage_random
        self.wafer_random = wafer_random
        self.grid_random = grid_random
        self.boundary_box = []

    def GenerateJson(self , name , topr_x , topr_y , botr_x , botr_y):
        self.boundary_box.append({
            "label": f"{name}",
            "points": [[int(topr_x), int(topr_y)], [int(botr_x), int(botr_y)]],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        })

    def Create_Stage(self):
        x1 = 0
        y1 = 0
        variation = math.floor((self.stage_random)/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:
                if ((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.stage_radius ** 2):
                    self.imgarr[x1][y1] = self.stage_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Wafer_Edge(self):
        x1 = 0
        y1 = 0
        variation = math.floor(self.wafer_random/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:

                if ((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2):
                    self.imgarr[x1][y1] = self.wafer_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Notch(self):
        x1 = 0
        y1 = 0
        variation = math.floor(self.stage_random/2)
        for x in self.imgarr:
            y1 = 0
            for y in x:
                if ((x1-(self.center_x + self.wafer_radius)) ** 2 + (y1-self.center_y) ** 2) <= (self.notch_radius ** 2):
                    self.imgarr[x1][y1] = self.stage_color + random.randint(-variation, variation)
                y1 = y1 +1
            x1 = x1 + 1

    def Create_Grid(self):
        half_scribe_x = math.floor(self.scribe_x/2)
        half_scribe_y = math.floor(self.scribe_y/2)
        xi = 0
        x=0
        x_i = 0
        y=0
        y_i = 0
        variation = math.floor(self.grid_random/2)
        for x1 in self.imgarr:
            y = 0
            y_i = 0
            for y1 in x1:
                if (x == x_i):
                    #print("found_x")
                    xi = 1
                    if ((x-self.center_x) ** 2 + (y-self.center_y) ** 2) <= (self.grid_radius ** 2):
                        #print("found")
                        temp_x = x-half_scribe_x
                        while temp_x <= x+half_scribe_x:
                            self.imgarr[temp_x][y] = self.grid_color + random.randint(-variation, variation)
                            temp_x += 1
                if(y == y_i):
                    #print("found_y")
                    y_i += self.pitch_y
                    if ((x-self.center_x) ** 2 + (y-self.center_y) ** 2) <= (self.grid_radius ** 2):
                        #print("found")
                        temp_y = y-half_scribe_y
                        while temp_y <= y+half_scribe_y:
                            self.imgarr[x][temp_y] = self.grid_color + random.randint(-variation, variation)
                            temp_y += 1
                y += 1
            x += 1
            if(xi):
                x_i += self.pitch_x
                xi = 0

    def create_img(self):
        self.Create_Stage()
        self.Create_Wafer_Edge()
        self.Create_Grid()
        self.Create_Notch()
        self.boundary_box = []
        #print("Finish Wafer")

    def point_in_circle(self , center_x , center_y , radius):
        while True:
            x = np.random.randint(center_x - radius, center_x + radius + 1)
            y = np.random.randint(center_y- radius, center_y + radius + 1)
            if (x - center_x)**2 + (y - center_y)**2 <= radius**2:
                return x, y

    def DrawBoundary(self ,min_x , min_y , max_x , max_y , name , min_box_size):
        if name != 'Cloudy':
            width = abs(max_x - min_x)
            height = abs(max_y - min_y)
            if width < min_box_size:
                diff = min_box_size - width
                min_x -= diff // 2
                max_x += diff // 2
            if height < min_box_size:
                diff = min_box_size - height
                min_y -= diff // 2
                max_y += diff // 2
            #cv.rectangle(self.imgarr, (min_x - 2 , min_y - 2), (max_x + 2 , max_y + 2), (0, 0, 255), 1)
            self.GenerateJson(name=name , topr_x=min_x , topr_y= min_y , botr_x=max_x , botr_y= max_y)
            #print(f"Finish Boundary for {name}")
        else:
            #cv.rectangle(self.imgarr, (min_x - 2 , min_y - 2), (max_x + 2 , max_y + 2), (0, 0, 255), 1)
            self.GenerateJson(name=name , topr_x=min_x , topr_y= min_y , botr_x=max_x , botr_y= max_y)
            #print(f"Finish Boundary for {name}")

    def GenerateScratch(self, name, min_box_size, variance_add, scratch_variance, scratch_length, scratch_thickness,):
        x1, y1 = self.point_in_circle(self.center_x, self.center_y, self.grid_radius) # Check point in the circle
        angle = np.random.uniform(0, 2 * np.pi) # Generate a random

        scratch_thickness = scratch_thickness - 1

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

        # Compute the x and y offsets using the angle and desired scratch length
        x2 = x1 + int(scratch_length * np.cos(angle))
        y2 = y1 + int(scratch_length * np.sin(angle))

        x = np.linspace(x1, x2, scratch_length, dtype=int) # Generate x array from start to end within the length
        y = np.linspace(y1, y2, scratch_length, dtype=int) # Generate y array from start to end within the length

        check_coords = [(ix, iy) for ix in range(-scratch_thickness, scratch_thickness + 1)
                            for iy in range(-scratch_thickness, scratch_thickness + 1)
                        if np.sqrt(ix**2 + iy**2) <= scratch_thickness]


        for ix, iy in check_coords:
            x_check_coords = x + ix
            y_check_coords = y + iy
            valid_coords = (x_check_coords >= 0) & (x_check_coords < self.img_size) & (y_check_coords >= 0) & (y_check_coords < self.img_size) & (((x_check_coords-self.center_x) ** 2 + (y_check_coords-self.center_y) ** 2) <= (self.wafer_radius ** 2))
            x_check_coords, y_check_coords = x_check_coords[valid_coords], y_check_coords[valid_coords]

        for i in range(max(len(x_check_coords), len(y_check_coords))):
            if(variance_add == 2):
                self.imgarr[x_check_coords[i], y_check_coords[i]] = self.imgarr[x_check_coords[i], y_check_coords[i]] + random.randint(math.floor(scratch_variance/2), scratch_variance)
                if(self.imgarr[x_check_coords[i], y_check_coords[i]] > 255):
                    self.imgarr[x_check_coords[i], y_check_coords[i]] = 255
                elif(self.imgarr[x_check_coords[i], y_check_coords[i]] < 0):
                    self.imgarr[x_check_coords[i], y_check_coords[i]] = 0
            else:
                self.imgarr[x_check_coords[i], y_check_coords[i]] = self.imgarr[x_check_coords[i], y_check_coords[i]] - random.randint(math.floor(scratch_variance/2), scratch_variance)
                if(self.imgarr[x_check_coords[i], y_check_coords[i]] > 255):
                    self.imgarr[x_check_coords[i], y_check_coords[i]] = 255
                elif(self.imgarr[x_check_coords[i], y_check_coords[i]] < 0):
                    self.imgarr[x_check_coords[i], y_check_coords[i]] = 0
            found = 1
            min_x = min(min_x, y_check_coords[i])
            max_x = max(max_x, y_check_coords[i])
            min_y = min(min_y, x_check_coords[i])
            max_y = max(max_y, x_check_coords[i])

        # Boundary Box
        #min_x = min(x) - scratch_thickness
        #min_y = min(y) - scratch_thickness
        #max_x = max(x) + scratch_thickness
        #max_y = max(y) + scratch_thickness
        if found == 1:
            self.DrawBoundary(min_x, min_y, max_x, max_y, name, min_box_size)
            return 1
        return 0

    def Cloudy_defect(self, min_r, max_r, set_radius_x, set_radius_y, min_box_size, name, blurriness, cloudy_variance, variance_add, cloudy_boundary, check_inside):
        start_cloud = time.time()
        x_radius = set_radius_x or random.randint(min_r, max_r)
        y_radius = set_radius_y or random.randint(min_r, max_r)

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

        if (x_radius > self.wafer_radius or y_radius > self.wafer_radius):
            return 0

        while True:
            rand_x = random.randint(int(self.center_x - self.wafer_radius), int(self.center_x + self.wafer_radius))
            rand_y = random.randint(int(self.center_y - self.wafer_radius ), int(self.center_y + self.wafer_radius ))

            if(check_inside == True):
                break

            #Check if the entire ellipse lies within the wafer_radius
            #if all((((rand_x + dx) - self.center_x) ** 2 + ((rand_y + dy) - self.center_y) ** 2) <= (self.wafer_radius-cloudy_boundary) ** 2
            #for dx in [-x_radius, x_radius] for dy in [-y_radius, y_radius]):

            if((rand_x - self.center_x)**2 + (rand_y - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                if((rand_x - self.center_x)**2 + (rand_y + y_radius - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                    if((rand_x + x_radius - self.center_x)**2 + (rand_y + y_radius - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                        if((rand_x + x_radius - self.center_x)**2 + (rand_y - self.center_y)**2 <= (self.wafer_radius - cloudy_boundary)**2):
                            break

        mask = np.zeros_like(self.imgarr)
        # Draw a ellipse on the mask.
        #cv.ellipse(mask, (rand_x, rand_y), (x_radius, y_radius), 0, 0, 360, 255, -1)
        if(GPUtil.getAvailable()):
          cv.rectangle(np.asnumpy(mask), (rand_x, rand_y), (rand_x + x_radius, rand_y + y_radius), 255, -1)
        else:
          cv.rectangle(mask, (rand_x, rand_y), (rand_x + x_radius, rand_y + y_radius), 255, -1)

        x1 = 0
        y1 = 0
        for x in mask:
            y1 = 0
            for y in x:
                if (not (((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2))):
                    mask[x1, y1] = 0
                elif((((x1-self.center_x) ** 2 + (y1-self.center_y) ** 2) <= (self.wafer_radius ** 2))):
                    if(mask[x1,y1] == 255):
                        if(variance_add == 2):
                            self.imgarr[x1, y1] = self.imgarr[x1, y1] + random.randint(math.floor(cloudy_variance/2), cloudy_variance)
                            if(self.imgarr[x1, y1] > 255):
                                self.imgarr[x1, y1] = 255
                            elif(self.imgarr[x1, y1] < 0):
                                self.imgarr[x1, y1] = 0

                        else:
                            self.imgarr[x1, y1] = self.imgarr[x1, y1] - random.randint(math.floor(cloudy_variance/2), cloudy_variance)
                            if(self.imgarr[x1, y1] > 255):
                                self.imgarr[x1, y1] = 255
                            elif(self.imgarr[x1, y1] < 0):
                                self.imgarr[x1, y1] = 0

                        found = 1
                        min_x = min(min_x, y1)
                        max_x = max(max_x, y1)
                        min_y = min(min_y, x1)
                        max_y = max(max_y, x1)

                y1 = y1 +1
            x1 = x1 + 1

        if(found == 0):
            return 0

        if(blurriness % 2 == 0):
            blurriness += 1
        # Apply the Gaussian blur
        for i in range(1,blurriness,2):
            blurred_image = cv.GaussianBlur(self.imgarr, (i, i), 0)

        # Combine original and blurred images based on the mask.
        self.imgarr = cv.bitwise_and(self.imgarr, self.imgarr, mask=~mask)
        blurred_region = cv.bitwise_and(blurred_image, blurred_image, mask=mask)
        self.imgarr = cv.add(self.imgarr, blurred_region)

        end_cloud = time.time()

        print("CLoud dy Defects end Time : " , start_cloud - end_cloud)

        # Draw the bounding box.
        #box_width = max(2 * x_radius, min_box_size)
        #box_height = max(2 * y_radius, min_box_size)
        #top_left = (int(rand_x - box_width / 2), int(rand_y - box_height / 2))
        #bottom_right = (int(rand_x + box_width / 2), int(rand_y + box_height / 2))
        if found == 1:
            min_x = min_x -5
            min_y = min_y -5
            max_x = max_x +5
            max_y = max_y +5
            self.DrawBoundary(min_x , min_y , max_x, max_y, name, min_box_size)
            return 1
        return 0

    def GenerateSpotDefects(self, name, min_box_size, spot_variance, spot_radius, variance_add):
        x, y = self.point_in_circle(self.center_x, self.center_y, self.grid_radius)

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        found = 0

    # Now, create a spot defect at the given x, y with the desired radius.
        for dx in range(-spot_radius, spot_radius + 1):
            for dy in range(-spot_radius, spot_radius + 1):
                if dx**2 + dy**2 <= spot_radius**2:  # This ensures that we only draw inside the circle.
                    x_coord, y_coord = x + dx, y + dy
                    if (0 <= x_coord < self.img_size) and (0 <= y_coord < self.img_size):  # Ensure the coordinates are inside the image.
                        if(((x_coord-self.center_x) ** 2 + (y_coord-self.center_y) ** 2) <= (self.wafer_radius ** 2)):
                            if(variance_add == 2):
                                self.imgarr[x_coord][y_coord] = self.imgarr[x_coord][y_coord] + random.randint(math.floor(spot_variance/2), spot_variance)
                                if(self.imgarr[x_coord][y_coord] > 255):
                                    self.imgarr[x_coord][y_coord] = 255
                                elif(self.imgarr[x_coord][y_coord] < 0):
                                    self.imgarr[x_coord][y_coord] = 0
                            else:
                                self.imgarr[x_coord][y_coord] = self.imgarr[x_coord][y_coord] - random.randint(math.floor(spot_variance/2), spot_variance)
                                if(self.imgarr[x_coord][y_coord] > 255):
                                    self.imgarr[x_coord][y_coord] = 255
                                elif(self.imgarr[x_coord][y_coord] < 0):
                                    self.imgarr[x_coord][y_coord] = 0

                            found = 1
                            min_x = min(min_x, y_coord)
                            max_x = max(max_x, y_coord)
                            min_y = min(min_y, x_coord)
                            max_y = max(max_y, x_coord)
        # Boundary Box
        if found == 1:
            self.DrawBoundary(min_x , min_y , max_x, max_y, name , min_box_size)
            return 1
        return 0

    def GenerateRadial(self, name, min_box_size, radius, angle1, variance, variance_add):


        x1, y1 = self.point_in_circle(self.center_x, self.center_y, self.grid_radius)

        angle1 = math.floor(angle1/2)

        min_x = self.img_size
        max_x = 0
        min_y = self.img_size
        max_y = 0
        max_radius = self.grid_radius
        found =0

        # Set the triangle's height as a random value
        #triangle_height = random.uniform(0.1 * max_radius, 0.4 * max_radius)
        # Random angles for the radial defect
        #angle_start = random.uniform(0, 2 * np.pi)
        #angle_end = angle_start + ((angle1 * np.pi)/180)

        angle_help = np.arctan2((x1 - self.center_x), (y1 - self.center_y)) % (2 * np.pi)

        angle_start = angle_help - ((angle1 * np.pi)/180)
        angle_end = angle_help + ((angle1 * np.pi)/180)

        for x in range(self.img_size):
            for y in range(self.img_size):
                angle = np.arctan2(x - x1, y - y1) % (2 * np.pi) # find angle pixel respect to the center within 0 to 2pi
                # Calculate the maximum distance for a point to be considered within the triangle
                if angle_start <= angle <= angle_end:     # check if angle pixel lie within the angle
                    #triangle_ratio = (angle - angle_start) / (angle_end - angle_start) # How far triangle span
                    #max_distance = triangle_ratio * triangle_height    #maximum distance fro mthe center that pixel can be
                    #distance_to_edge = max_radius - np.sqrt((x - self.center_x)**2 + (y - self.center_y)**2) # radial distance from the center
                    #if 0 <= distance_to_edge <= max_distance:   # check if the pixel inside the triangle
                    #    # With a certain probability, fill the spot
                    new_x = x
                    new_y = y
                    if((new_x-x1) ** 2 + (new_y-y1) ** 2) <= (radius ** 2):

                        # Ensure new_x and new_y are within image boundaries
                        if ((new_x-self.center_x) ** 2 + (new_y-self.center_y) ** 2) <= (self.wafer_radius ** 2):
                            if (variance_add == 2):
                                self.imgarr[new_x, new_y] = self.imgarr[new_x, new_y] + random.randint(math.floor(variance/2), variance)
                                if(self.imgarr[new_x, new_y] > 255):
                                    self.imgarr[new_x, new_y] = 255
                                elif(self.imgarr[new_x, new_y] < 0):
                                    self.imgarr[new_x, new_y] = 0

                            else:
                                self.imgarr[new_x, new_y] = self.imgarr[new_x, new_y] - random.randint(math.floor(variance/2), variance)
                                if(self.imgarr[new_x, new_y] > 255):
                                    self.imgarr[new_x, new_y] = 255
                                elif(self.imgarr[new_x, new_y] < 0):
                                    self.imgarr[new_x, new_y] = 0

                            found = 1
                            min_x = min(min_x, new_y)
                            max_x = max(max_x, new_y)
                            min_y = min(min_y, new_x)
                            max_y = max(max_y, new_x)
        if found==1:
            self.DrawBoundary(min_x , min_y  , max_x, max_y, name= name, min_box_size=min_box_size)
            return 1
        return 0


    def GenerateRadialDefect2(self, defect_length, thickness, num_lines_per_defect, spacing, line_height_scale , min_box_size):
        base_angle = np.random.uniform(0, 2 * np.pi)
        min_x, max_x, min_y, max_y = float('inf'), float('-inf'), float('inf'), float('-inf')  # Initialize min and max values

        for j in range(num_lines_per_defect):
            # Modify base_angle with a fixed spacin
            angle = base_angle + j * spacing

            # caculate line height
            length_offset = int(defect_length * np.random.uniform(-line_height_scale, line_height_scale))
            adjusted_defect_length = defect_length + length_offset

            x_start = self.center_x + int(self.grid_radius * np.cos(angle))
            y_start = self.center_y + int(self.grid_radius * np.sin(angle))

            x_end = x_start - int(adjusted_defect_length * np.cos(angle))
            y_end = y_start - int(adjusted_defect_length * np.sin(angle))

            x = np.linspace(x_start, x_end, adjusted_defect_length, dtype=int)
            y = np.linspace(y_start, y_end, adjusted_defect_length, dtype=int)

            self.imgarr[y, x] = np.random.randint(50, 90, size=x.shape)

            # Generating main line coordinates (with reduced thickness)
            line_coords = [(ix, iy) for ix in range(-thickness, thickness + 1)
                for iy in range(-thickness, thickness + 1)
                if np.sqrt(ix**2 + iy**2) <= thickness]

            for ix, iy in line_coords:
                x_line_coords = x + ix
                y_line_coords = y + iy
                valid_coords = (x_line_coords >= 0) & (x_line_coords < self.img_size) & (y_line_coords >= 0) & (y_line_coords < self.img_size)
                x_line_coords, y_line_coords = x_line_coords[valid_coords], y_line_coords[valid_coords]
                self.imgarr[y_line_coords, x_line_coords] = np.random.randint(50, 90, size=x_line_coords.shape)

            min_x = min(min_x, min(x))
            max_x = max(max_x, max(x))
            min_y = min(min_y, min(y))
            max_y = max(max_y, max(y))

        self.DrawBoundary(min_x , min_y  , max_x, max_y, name='Radial2' , min_box_size=min_box_size)

    def save_img(self , filename):
        if(GPUtil.getAvailable()):
          cv.imwrite(filename, np.asnumpy(self.imgarr))
        else:
          cv.imwrite(filename, self.imgarr)

    def save_json(self , filename):
        data = {
            "version": "5.0.1",
            "flags": {},
            "shapes": self.boundary_box,
            "imagePath": filename,
            "imageData": None,
            "imageHeight": self.img_size,
            "imageWidth": self.img_size
        }
        json_file = os.path.join('img_data/', os.path.basename(filename).split('.')[0] + '.json')
        with open(json_file, 'w') as json_file:
            json.dump(data, json_file)

###################################################################################


cupy


In [ ]:
#from GenerateData import GenerateData
from PIL import Image
#import numpy as np
import random
import time


def save_base(gr , filename):
    gr.create_img()
    gr.save_img(filename)

if __name__ == '__main__':
    ##different paramters to change:

    number_of_images = 1
    number_of_reuse = 1 #how images will share the same background

    img_size= 601
    min_box_size = 20




    radial_defect_name = 'radial'
    spot_defect_name =  'spot'
    cloudy_defect_name = 'cloudy'
    line_defect_name = 'scratch'

    set_chip_color = None ##if you want to set the color directly
    set_stage_color = None ##if you want to set the color directly
    set_grid_color = None ##if you want to set the color directly
    set_grid_difference = None ## how much brighter or darker the grid is to the wafer
    set_grid_darker = None ##set to 1 if you want to make grid darker than wafer and 2 to make grid brighter otherwise random
    set_pitch_x = None ##if you want to set how big the grid is directly
    set_pitch_y = None ##if you want to set how big the grid is directly
    set_scribe_x = None ##if you want to set how thick the grid lines are
    set_scribe_y = None ##if you want to set how thick the grid lines are
    set_number_defects = None ##if you want to set how many defects per image
    set_cloudy_radius_x = None ##if you want to set the radius of the cloudy circle/oval
    set_cloudy_radius_y = None ##if you want to set the radius of the cloudy circle/oval
    set_cloudy_variance = None
    set_cloudy_variance_add = None ##set to 1 to make scratchs darker than base, 2 to make scratchs lighter than base
    set_cloudy_boundary = None ##how many pixels should the edge of any cloudy oval be from the wafer_edge
    set_scratch_length = None
    set_scratch_thickness = None
    set_scratch_variance = None
    set_scratch_variance_add = None ##set to 1 to make scratchs darker than base, 2 to make scratchs lighter than base
    set_spot_variance = None
    set_spot_radius = None
    set_spot_variance_add = None ##set to 1 to make spot darker than base, 2 to make scratchs lighter than base
    set_radial_radius = None
    set_radial_variance = None
    set_radial_angle = None
    set_radial_variance_add = None ##set to 1 to make radial darker than base, 2 to make scratchs lighter than base

    defect_min = 1
    defect_max = 5

    cloudy_check_inside = False ## set to true to check whether the entire clody rectangle is inside the wafer or not

    chip_low = 70
    chip_high = 200
    stage_low = 20
    stage_high = 40
    min_grid_difference = 10
    max_grid_difference = 20
    pitch_min = 10
    pitch_max = 15
    scribe_min = 1
    scribe_max = 3
    stage_variation_low = 5
    stage_variation_high = 10
    wafer_variation_low = 5
    wafer_variation_high = 10
    grid_variation_low = 2
    grid_variation_high = 5

    scratch_length_min = 10
    scratch_length_max = 20
    scratch_thickness_min = 1
    scratch_thickness_max = 2
    scratch_variance_min = 30
    scratch_variance_max = 40

    spot_radius_min = 1
    spot_radius_max = 3
    spot_variance_min = 20
    spot_variance_max = 30

    cloudy_radius_min = 5
    cloudy_radius_max = 20
    cloudy_blurriness = 5 ##should only be odd but no error just adds one
    min_cloudy_variance = 20
    max_cloudy_variance = 30
    min_cloudy_boundary = 3
    max_cloudy_boundary = 7

    radial_variance_min = 30
    radial_variance_max = 40
    radial_angle_min = 1
    radial_angle_max = 10
    radial_radius_min = 10
    radial_radius_max = 40


    #end of changeable parameters





    stage_mid_color = set_stage_color or random.randint(stage_low, stage_high)
    wafer_mid_color = set_chip_color or random.randint(chip_low, chip_high)
    grid_darker = set_grid_darker or random.randint(1,2)
    grid_difference = set_grid_difference or random.randint(min_grid_difference, max_grid_difference)
    if(grid_darker == 2):
        grid_mid_color = set_grid_color or (wafer_mid_color + grid_difference)
    else:
        grid_mid_color = set_grid_color or (wafer_mid_color - grid_difference)

    pitch_x = set_pitch_x or random.randint(pitch_min, pitch_max)
    pitch_y = set_pitch_y or random.randint(pitch_min, pitch_max)
    scribe_x = set_scribe_x or random.randint(scribe_min, scribe_max)
    scribe_y = set_scribe_y or random.randint(scribe_min, scribe_max)

    stage_random = random.randint(stage_variation_low, stage_variation_high)
    wafer_random = random.randint(wafer_variation_low, wafer_variation_high)
    grid_random = random.randint(grid_variation_low, grid_variation_high)


    gr = GenerateData(
        img_size,
        chip_low,
        chip_high,
        stage_mid_color,
        wafer_mid_color,
        grid_mid_color,
        stage_random,
        wafer_random,
        grid_random,
        pitch_x,
        pitch_y,
        scribe_x,
        scribe_y,

    )

    save_base(gr , "base_img/baseimg.jpg")

    for i in range(number_of_images):

        start = time.time()

        if(i % number_of_reuse == 0):
            stage_mid_color = set_stage_color or random.randint(stage_low, stage_high)
            wafer_mid_color = set_chip_color or random.randint(chip_low, chip_high)
            grid_darker = set_grid_darker or random.randint(1,2)
            grid_difference = set_grid_difference or random.randint(min_grid_difference, max_grid_difference)
            if(grid_darker == 2):
                grid_mid_color = set_grid_color or (wafer_mid_color - grid_difference)
            else:
                grid_mid_color = set_grid_color or (wafer_mid_color + grid_difference)

            pitch_x = set_pitch_x or random.randint(pitch_min, pitch_max)
            pitch_y = set_pitch_y or random.randint(pitch_min, pitch_max)
            scribe_x = set_scribe_x or random.randint(scribe_min, scribe_max)
            scribe_y = set_scribe_y or random.randint(scribe_min, scribe_max)

            stage_random = random.randint(stage_variation_low, stage_variation_high)
            wafer_random = random.randint(wafer_variation_low, wafer_variation_high)
            grid_random = random.randint(grid_variation_low, grid_variation_high)

            gr = GenerateData(
            img_size,
            chip_low,
            chip_high,
            stage_mid_color,
            wafer_mid_color,
            grid_mid_color,
            stage_random ,
            wafer_random,
            grid_random,
            pitch_x,
            pitch_y,
            scribe_x,
            scribe_y,)

        gr.create_img()
        defect = set_number_defects or random.randint(defect_min, defect_max)


        for z in range(defect):

            cloudy_variance_add = set_cloudy_variance_add or random.randint(1,2)
            cloudy_variance = set_cloudy_variance or random.randint(min_cloudy_variance, max_cloudy_variance)
            cloudy_boundary = set_cloudy_boundary or random.randint(min_cloudy_boundary, max_cloudy_boundary)

            scratch_varince_add = set_scratch_variance_add or random.randint(1, 2)
            scratch_variance = set_scratch_variance or random.randint(scratch_variance_min, scratch_variance_max)
            scratch_length = set_scratch_length or random.randint(scratch_length_min, scratch_length_max)
            scratch_thickness = set_scratch_thickness or random.randint(scratch_thickness_min,scratch_thickness_max)

            spot_variance = set_spot_variance or random.randint(spot_variance_min, spot_variance_max)
            spot_radius = set_spot_radius or random.randint(spot_radius_min, spot_radius_max)
            spot_variance_add = set_spot_variance_add or random.randint(1,2)

            radial_radius = set_radial_radius or random.randint(radial_radius_min, radial_radius_max)
            radial_angle = set_radial_angle or random.randint(radial_angle_min, radial_angle_max)
            radial_variance = set_radial_variance or random.randint(radial_variance_min, radial_variance_max,)
            radial_variance_add = set_radial_variance_add or random.randint(1,2)


            random_defect = random.randint(1,4)
            if (random_defect == 1):
                done = gr.GenerateScratch(line_defect_name, min_box_size, scratch_varince_add, scratch_variance, scratch_length, scratch_thickness)
            if (random_defect == 2):
                done = gr.GenerateSpotDefects(spot_defect_name, min_box_size, spot_variance, spot_radius, spot_variance_add)
            # if (random_defect == 3):
            #     done = gr.Cloudy_defect(cloudy_radius_min , cloudy_radius_max, set_cloudy_radius_x, set_cloudy_radius_y, min_box_size, cloudy_defect_name, cloudy_blurriness, cloudy_variance, cloudy_variance_add, cloudy_boundary, cloudy_check_inside)
            if (random_defect == 4):
                done = gr.GenerateRadial(radial_defect_name, min_box_size, radial_radius, radial_angle, radial_variance, radial_variance_add)
                # gr.GenerateRadialDefect2(min_box_size = min_box_size ,defect_length=random.randint(30, 50) , thickness=random.randint(1,5) , num_lines_per_defect=random.randint(2,5) , spacing=random.uniform(0.03 , 0.05) , line_height_scale=random.uniform(0.1,0.7))
            if(done == 0):
                defect = defect - 1
            end = time.time()
        print("Image takes" + f'{end - start} seconds')


        #gr.GenerateScratch(line_defect_name, min_box_size, scratch_varince_add, scratch_variance, scratch_length, scratch_thickness)
        #gr.GenerateSpotDefects(spot_defect_name, min_box_size, spot_variance, spot_radius, spot_variance_add)
        #gr.Cloudy_defect(cloudy_radius_min , cloudy_radius_max, set_cloudy_radius_x, set_cloudy_radius_y, min_box_size, cloudy_defect_name, cloudy_blurriness)
        #gr.GenerateRadial(radial_defect_name, min_box_size, radial_radius, radial_angle, radial_variance, radial_variance_add)
        gr.save_img(filename= f'img_data/test{i+1}.jpg')
        #gr.save_json(filename= f'test{i+1}.jpg')
        print("done")